## Section I

Group By

In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [2]:
df_green = spark.read.parquet('D:/data/pq/green/*/*')

In [3]:
df_green.registerTempTable('green')

C:\Users\balaj\anaconda3\lib\site-packages\pyspark\sql\dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


In [6]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY 
    1, 2
""")

In [7]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   7| 769.7299999999997|            45|
|2020-01-01 00:00:00|  17|195.03000000000006|             9|
|2020-01-01 00:00:00|  18|               7.8|             1|
|2020-01-01 00:00:00|  22|              15.8|             1|
|2020-01-01 00:00:00|  24|              87.6|             3|
|2020-01-01 00:00:00|  25| 531.0000000000001|            26|
|2020-01-01 00:00:00|  29|              61.3|             1|
|2020-01-01 00:00:00|  32| 68.94999999999999|             2|
|2020-01-01 00:00:00|  33|317.27000000000004|            11|
|2020-01-01 00:00:00|  35|            129.96|             5|
|2020-01-01 00:00:00|  36|            295.34|            11|
|2020-01-01 00:00:00|  37|            175.67|             6|
|2020-01-01 00:00:00|  38| 98.78999999999999|             2|
|2020-01-01 00:00:00|  4

In [13]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('D:data/report/revenue/green', mode='overwrite')

In [14]:
df_yellow = spark.read.parquet('D:/data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [15]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [16]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('D:/data/report/revenue/yellow', mode='overwrite')

## Section II

Joins

Type 1 - Tables of equal/similar size

In [17]:
df_green_revenue = spark.read.parquet('D:/data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('D:/data/report/revenue/yellow')

In [18]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [20]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [21]:
df_join.write.parquet('D:/data/report/revenue/total', mode='overwrite')

In [22]:
df_join = spark.read.parquet('D:/data/report/revenue/total')

In [23]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

Type 2 - Big table and a smaller table

In [48]:
import os

directory = "C:/users/balaj/zones"
if not os.path.exists(directory):
    os.makedirs(directory)
    
!curl -L -o "C:/users/balaj/zones/taxi_zone_lookup.csv" "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv"

find: '/I': No such file or directory
find: '/N': No such file or directory
find: 'SoundMixer.exe': No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 12322  100 12322    0     0  16245      0 --:--:-- --:--:-- --:--:-- 56522


In [60]:
df_temp = spark.read.csv("C:/users/balaj/zones/taxi_zone_lookup.csv", header=True)

In [62]:
df_temp.write.parquet("C:/users/balaj/zones/taxi_zone_lookup")

In [63]:
df_zones = spark.read.parquet('C:/users/balaj/zones/taxi_zone_lookup/')

In [64]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [65]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')